In [ ]:
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install scipy
!pip install scikit-learn
!pip install statsmodels
!pip install seaborn
!pip install numpy-indexed
!pip install pingouin
!pip install ggpubr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import scipy
import numpy.matlib

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
sns.set_theme(style="ticks", color_codes=True)
import glob

!pip install numpy-indexed
import numpy_indexed as npi
from sklearn.linear_model import LinearRegression

In [ ]:
# Get CSV files list 

csvFilenamesList = glob.glob('*LinearPI_MainTaskLog.csv')
csvFilenamesList.sort()

csvFilenamesList.pop(0) # remove the first one  sub-00- from the previous pilot
print(len(csvFilenamesList))

csvFilenamesList

In [ ]:
# Read each CSV file into DataFrame. This creates a list of dataframes

df_list=[]
for f in csvFilenamesList:
    temp_df = pd.read_csv(f, sep=None, engine='python')
    temp_df
    df_list.append (temp_df)
    
mean_dist = round(np.mean(df_list[1]['mainMeters'].unique()), 2)
mean_dist

In [ ]:
# computing the distance estimation error and its absolute value
df_list_ext=[]
for df in df_list:
    df_temp = df.assign(est_error=lambda x: (x.mainEstimate-x.mainMeters))
    df_temp = df_temp.assign(abs_est_error=lambda x: abs(x.mainEstimate-x.mainMeters))
    df_temp = df_temp.assign(center_dist=lambda x: abs(x.mainEstimate-mean_dist))
    df_list_ext.append(df_temp)

In [ ]:
# Finding the missed trials and removing them from the data frames of each participant

nSubjects=len(df_list_ext)
missedTrials=np.zeros((nSubjects,1))

df_no_missed_list = []
for df in df_list_ext:
    print(len(df))
    df_no_missed = df[df['mainEstimate'] != -1].copy()
    df_no_missed_list.append(df_no_missed)
    print(len(df_no_missed))
    print('\n')


In [ ]:
# Printing the range of real distances for each participant
main_dist = df_no_missed_list[1]['mainMeters'].unique()
print('Main Meters =', main_dist)

trial_duration = df_no_missed_list[1]['mainDuration'].unique()
print('Main Duration = ', trial_duration)
duration_min = np.min(trial_duration)
duration_max = np.max(trial_duration)
print(duration_min, duration_max)

speed_level=df_no_missed_list[1]['mainSpeed'].unique() 
print('Speed levels = ', speed_level)

xAcc=df_no_missed_list[1]['mainAccelerationDuration'].unique()  
xDec=df_no_missed_list[1]['mainDecelerationDuration'].unique()  
accdec_dur=.5*(np.unique(xAcc)+numpy.unique(xDec));
print('\n')

In [ ]:
# removing the catch trials 

df_no_missed_no_catch_list = []
for df in df_no_missed_list:
    df_no_missed_no_catch = df[(df['mainDuration'] != duration_min) & (df['mainDuration'] != duration_max)].copy()    
    df_no_missed_no_catch_list.append(df_no_missed_no_catch)
    print(len(df_no_missed_no_catch))


In [ ]:
# adding IDs
sn=1
for df in df_no_missed_no_catch_list:
    [a, b]=(df).shape
    if sn < 10:
        df["ID"]=(['P0' + str(sn)]*a)
    else:
        df["ID"]=(['P' + str(sn)]*a)
    sn=sn+1

In [ ]:
# Concatinating dfs
df_concat=pd.concat(df_no_missed_no_catch_list)
df_concat

In [ ]:
# Plotting the regression fitted line for real vs. estimated distances for each ptp

for df_temp in df_no_missed_no_catch_list:
    X = df_temp.iloc[:,10].values.reshape(-1, 1)  # real distnces 
    Y = df_temp.iloc[:,9].values.reshape(-1, 1)  # estimated distances 
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    Y_pred = linear_regressor.predict(X)

    plt.scatter(X, Y)
    plt.plot(X, Y_pred, color='red', label='fitted line')
    plt.plot(X, X, color='green', label='perfect estimate')
    plt.legend(loc="upper left")

    plt.title(df_temp.iloc[1,21] , fontsize=15)
    plt.xlabel("Real Distances", fontsize=15)
    plt.ylabel("Estimated Distances", fontsize=15)
    plt.xlim([100, 500])
    plt.ylim([-20, 900])
    plt.savefig('./Figures/' + 'regfit_estimations_' + df_temp.iloc[1,21] + '.png', bbox_inches='tight')
    plt.show()


In [ ]:
print(df_temp.iloc[1,21])

In [ ]:
# Plotting the regression fitted line for estimation error for each ptp

for df_temp in df_no_missed_no_catch_list:
    X = df_temp.iloc[:,10].values.reshape(-1, 1)  # real distnces 
    Y = df_temp.iloc[:,18].values.reshape(-1, 1)  # estimation error 
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    Y_pred = linear_regressor.predict(X)

    zeroY=np.zeros(len(X))
    plt.scatter(X, Y)
    plt.plot(X, Y_pred, color='red', label='fitted line')
    plt.plot(X, zeroY, color='green', label='perfect estimate')
    plt.legend(loc="upper left")

    plt.title(df_temp.iloc[1,21] , fontsize=15)
    plt.xlabel("Real distances", fontsize=15)
    plt.ylabel("Estimation error", fontsize=15)
    plt.xlim([100, 500])
    plt.ylim([-400, 400])
    plt.savefig('./Figures/' + 'regfit_estimation_error_' + df_temp.iloc[1,21] + '.png', bbox_inches='tight')
    plt.show()

In [ ]:
# estimation error across runs for all the participants

ax=sns.pointplot(x='RunNumber', y=(df_concat.est_error), hue='ID', errorbar='sd', data=df_concat, \
                  order=[1, 2, 3, 4], dodge=False, linestyle='-', err_kws={'linewidth': 1}, palette="mako")
sns.set(font_scale = 1)
plt.xlabel('run number', fontsize = 15)
plt.ylabel('estimation error', fontsize = 15)
sns.set_style("whitegrid", {'axes.grid' : False})
# ax2.get_legend().remove()
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), title='')
plt.title('Estimation error across the runs \n', fontsize = 18)
plt.savefig('./Figures/'+'ErrorAcrossRuns_AllParticipants.png', bbox_inches = "tight")
plt.show()

In [ ]:
# Group the data by participant ID and condition, and compute the mean of each group

# compute the mean of estimations for each real distance for each participant: df_mean
df_mean = df_concat.groupby(['ID', 'mainMeters'])['mainEstimate'].mean().reset_index()

sns.set_palette("rocket")
# Plot the fitted line for every subject without scatter plot points
for subject_id in df_concat['ID'].unique():
    subject_data = df_concat[df_concat['ID'] == subject_id]
    sns.regplot(x='mainMeters', y='mainEstimate', data=subject_data, ci=None, scatter=False, line_kws={'alpha': 0.2})

# Plot the fitted line to the data of all participants
sns.regplot(x='mainMeters', y='mainEstimate', data=df_mean, ci=None, scatter=True, color='#6B007B', line_kws={'linewidth': 3})
plt.legend([],[], frameon=False)
plt.plot([100, 500], [100, 500],'k', linestyle='dashed', linewidth=1.5)

sns.set(font_scale = 1)
plt.xlabel('actual distances', fontsize = 16)
plt.ylabel('estimated distances', fontsize = 16)
sns.set_style("whitegrid", {'axes.grid' : False})
plt.title('Regression to the mean effect \n', fontsize = 18)
plt.tight_layout()
plt.savefig('Figures/'+'estimated_distVreal_dist_across_group.png', bbox_inches = "tight", dpi=300)

plt.show()

In [ ]:
ax = sns.pointplot(x='RunNumber', y='est_error', hue='mainSpeed', data=df_concat, order=[1, 2, 3, 4], dodge=False, err_kws={'linewidth': 1}, palette="mako")
sns.set(font_scale = 1)
plt.xlabel('run', fontsize = 16)
plt.ylabel('estimation error', fontsize = 16)
sns.set_style("whitegrid", {'axes.grid' : False})
# ax2.get_legend().remove()
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), title='')
plt.title('Estimation error across runs at each speed level', fontsize = 13)
plt.savefig('Figures/'+'estimation_error_across_runs_based_speed.png', bbox_inches = "tight", dpi=300)
plt.show()

In [ ]:
# effect of run number on distance estimation error across participants 

# Number of runs to analyze
nRun = 4

# Initialize a matrix to store mean absolute estimation errors for each subject and run
error_mat = np.zeros((len(df_list_ext), nRun)) 

# Initialize a subject index
sn = 0

# Loop through each dataframe in the list
for df in df_list_ext:  
    # Create a copy of the current dataframe to avoid modifying the original
    df_temp = df.copy()
    
    # Loop through each run
    for iRun in range(nRun):  
        # Initialize mean estimation error
        meanEstimationError = 0
        
        # Filter the dataframe for the current run
        df_run = df_temp.loc[df['RunNumber'] == iRun + 1]
        
        # Calculate the mean of the absolute estimation errors for the current run
        meanEstimationError = np.mean(abs(df_run.abs_est_error))
        
        # Store the mean estimation error in the error matrix
        error_mat[sn][iRun] = meanEstimationError
    
    # Increment the subject index for the next dataframe
    sn = sn + 1

# Create a run number array
run_mat = np.array([1, 2, 3, 4])

# Repeat the run numbers for each subject and flatten the result into a 1D array
run_vec = np.matlib.repmat(run_mat, 1, len(df_list_ext))
run_vec = run_vec.flatten()

# Flatten the error matrix into a 1D array
error_vec = error_mat.reshape(np.prod(error_mat.shape), 1)
error_vec = error_vec.flatten()

# Create a subjects vector, repeating each subject number for each run
subjects_vec = np.repeat(np.arange(len(df_list_ext)) + 1, nRun)
subjects_vec = subjects_vec.reshape(nRun * len(df_list_ext), 1)
subjects_vec = subjects_vec.flatten()

# Create a DataFrame for ANOVA analysis with subjects, run numbers, and errors
df_run = pd.DataFrame({'Subjects': subjects_vec,
                       'RunNumber': run_vec,
                       'Error': error_vec}, index=np.arange(1, nRun * len(df_list_ext) + 1))

# Print the DataFrame to check the structure
print(df_run)

# Perform repeated measures ANOVA on the error data
print(AnovaRM(data=df_run, depvar='Error', subject='Subjects', within=['RunNumber']).fit())

In [ ]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import statsmodels.api as sm  # For statistical models
from statsmodels.formula.api import ols, mixedlm  # For Ordinary Least Squares and Mixed Linear Models
from statsmodels.stats.anova import AnovaRM  # For Repeated Measures ANOVA
from statsmodels.stats.multicomp import pairwise_tukeyhsd  # For Tukey's HSD test
from scipy.stats import ttest_rel, ttest_ind  # For paired and independent t-tests
from statsmodels.stats.multitest import multipletests  # For multiple testing correction
import pingouin as pg  # For statistical tests and data analysis

# Group the data by participant (ID) and condition (mainMeters), and calculate the standard deviation of the response (est_error) for each group
std_dev = df_concat.groupby(['ID', 'mainMeters'])['est_error'].std().reset_index()

# Pivot the data to create a matrix of standard deviations, with participants as rows and conditions as columns
std_dev_matrix = std_dev.pivot(index='ID', columns='mainMeters', values='est_error')

# Set the color palette for the plots to "rocket"
sns.set_palette("rocket")

# Create a violin plot to visualize the distribution of standard deviations of estimation errors by condition
ax = sns.violinplot(x='mainMeters', y='est_error', data=std_dev)

# Add point plot to the violin plot to show mean estimation errors with error bars representing standard deviation
sns.pointplot(x='mainMeters', y='est_error', hue='ID', errorbar='sd', data=std_dev,
              dodge=False, err_kws={'linewidth': 1}, linewidth=1, palette="rocket", ax=ax)

# Perform pairwise t-tests between conditions to assess differences in estimation errors
pairwise_res = pg.pairwise_tests(data=std_dev, dv='est_error', within='mainMeters', subject='ID')

# Loop through the results of pairwise tests to add significance indicators to the plot
for i in range(len(pairwise_res)):
    if pairwise_res['p-unc'][i] < 0.05:  # Check if the p-value is less than the significance level (0.05)
        # Calculate the x-coordinates for the lines indicating significant differences
        x_min = pairwise_res['A'][i]
        x_min = (x_min - 120) / 120  # Normalize x_min for plotting
        x_max = pairwise_res['B'][i]
        x_max = (x_max - 120) / 120  # Normalize x_max for plotting
        
        # Set the y-coordinate for the significance line
        y_line = 10 * i + 140
        p_val = round(pairwise_res['p-unc'][i], 2)  # Round p-value for display
        
        # Draw a horizontal line to indicate significant differences
        ax.hlines(y=y_line, xmin=x_min, xmax=x_max, color='k', linewidth=1)
        
        # Annotate the plot with the p-value
        ax.annotate("p = {}".format(p_val), xy=((x_min + x_max) * .5, y_line), 
                    ha='center', va='bottom', color='k', fontsize=10)

# Remove top and right spines from the plot for better aesthetics
sns.despine()

# Set the limits and labels for the plot
plt.title('Variance of estimation error increases \n with distance \n', fontsize=18)
plt.xlabel('actual distances', fontsize=16)
plt.ylabel('sd of estimation error', fontsize=16)

# Remove legend labels for clarity
ax.legend(labels=[])

# Save the figure to a file with high resolution
plt.savefig('./Figures/' + 'estimation_error_sd.png', bbox_inches="tight", dpi=300)

# Display the plot
plt.show()

# perform a repeated measures ANOVA on the standard deviation of the response for each condition
rm = AnovaRM(std_dev, 'est_error', 'ID', within=['mainMeters'])
res = rm.fit()
print(res.summary())

In [ ]:

# Initialize lists to store the means for each half
first_half_means = []
second_half_means = []

for df in df_no_missed_no_catch_list:
    # Select run1 data
    run1_data = df[df.iloc[:, 0] == 1]
    
    # Calculate the midpoint
    midpoint = len(run1_data) // 2
    
    # Split the DataFrame and calculate means
    first_half_mean = run1_data.iloc[:midpoint, 20].median()  # Column 21 is index 20
    second_half_mean = run1_data.iloc[midpoint:, 20].median()
    
    first_half_means.append(first_half_mean)
    second_half_means.append(second_half_mean)

# Create a DataFrame for plotting
plot_data = pd.DataFrame({
    'Half': ['First Half'] * 29 + ['Second Half'] * 29,
    'Mean': first_half_means + second_half_means
})

# Create the violin plot with individual data points
plt.figure(figsize=(12, 8))
sns.violinplot(x='Half', y='Mean', data=plot_data, inner=None)
sns.swarmplot(x='Half', y='Mean', data=plot_data, color='black', size=5)

# Add lines connecting dots from the same participants
for i in range(len(first_half_means)):
    plt.plot([0, 1], [first_half_means[i], second_half_means[i]], color='gray', alpha=0.3)
    
plt.title('Distribution of Means for First and Second Half of Run1 Data')
plt.xlabel('Half of Run1 Data')
plt.ylabel('Mean of Column 10')

# Add some jitter to the swarm plot to reduce overlapping
plt.xlim(-0.5, 1.5)
plt.show()


# Assuming you have already created the first_half_means and second_half_means lists

# Perform paired t-test
t_statistic, p_value = stats.ttest_rel(first_half_means, second_half_means)

# Calculate effect size (Cohen's d for paired samples)
d = (np.mean(first_half_means) - np.mean(second_half_means)) / np.std(np.array(first_half_means) - np.array(second_half_means))

# Print results
print(f"Paired t-test results:")
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")
print(f"Effect size (Cohen's d): {d}")

# Determine if the difference is significant (using alpha = 0.05)
alpha = 0.05
if p_value < alpha:
    print(f"The difference between the two halves is statistically significant (p < {alpha}).")
else:
    print(f"The difference between the two halves is not statistically significant (p >= {alpha}).")



In [ ]:
# mean RT across runs

# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import statsmodels.api as sm  # For statistical models
from statsmodels.formula.api import ols, mixedlm  # For Ordinary Least Squares and Mixed Linear Models
from statsmodels.stats.anova import AnovaRM  # For Repeated Measures ANOVA
from statsmodels.stats.multicomp import pairwise_tukeyhsd  # For Tukey's HSD test
from scipy.stats import ttest_rel, ttest_ind  # For paired and independent t-tests
from statsmodels.stats.multitest import multipletests  # For multiple testing correction
import pingouin as pg  # For statistical tests and data analysis

# Group the data by participant (ID) and condition (Run number), and calculate the mean of the response (RT) for each group
mean_RT = df_concat.groupby(['ID', 'RunNumber'])['mainEstimateRT'].mean().reset_index()

# Pivot the data to create a matrix of means, with participants as rows and conditions as columns
# mean_RT_matrix = mean_RT.pivot(index='ID', columns='RunNumber', values='mainEstimateRT')

# Set the color palette for the plots to "rocket"
sns.set_palette("rocket")

# Create a violin plot to visualize the distribution of means of RTs by condition
ax = sns.violinplot(x='RunNumber', y='mainEstimateRT', data=mean_RT)

# Add point plot to the violin plot to show mean RT with error bars representing standard deviation
sns.pointplot(x='RunNumber', y='mainEstimateRT', hue='ID', errorbar='sd', data=mean_RT,
              dodge=False, err_kws={'linewidth': 1}, linewidth=1, palette="rocket", ax=ax)

# Perform pairwise t-tests between conditions to assess differences in RTs
pairwise_res_mean_RT = pg.pairwise_tests(data=mean_RT, dv='mainEstimateRT', within='RunNumber', subject='ID')

# # Loop through the results of pairwise tests to add significance indicators to the plot
for i in range(len(pairwise_res_mean_RT)):
    if pairwise_res_mean_RT['p-unc'][i] < 0.05:  # Check if the p-value is less than the significance level (0.05)
        # Calculate the x-coordinates for the lines indicating significant differences
        x_min = pairwise_res_mean_RT['A'][i]
        x_min = (x_min - 1) / 1  # Normalize x_min for plotting
        x_max = pairwise_res_mean_RT['B'][i]
        x_max = (x_max - 1) / 1  # Normalize x_max for plotting
        
        # Set the y-coordinate for the significance line
        y_line =  0.25*i + 6
        p_val = round(pairwise_res_mean_RT['p-unc'][i], 2)  # Round p-value for display
        
        # Draw a horizontal line to indicate significant differences
        ax.hlines(y=y_line, xmin=x_min, xmax=x_max, color='k', linewidth=1)
        
        # Annotate the plot with the p-value
        ax.annotate("p = {}".format(p_val), xy=((x_min + x_max) * .5, y_line), 
                    ha='center', va='bottom', color='k', fontsize=10)

# Remove top and right spines from the plot for better aesthetics
sns.despine()

# Set the limits and labels for the plot
plt.title('mean of RT across runs\n', fontsize=18)
plt.xlabel('run number', fontsize=16)
plt.ylabel('mean RT', fontsize=16)

# Remove legend labels for clarity
ax.legend(labels=[])

# Save the figure to a file with high resolution
plt.savefig('./Figures/' + 'mean_RT_across_runs.png', bbox_inches="tight", dpi=300)

# Display the plot
plt.show()

# perform a repeated measures ANOVA on the mean RT of the response for each condition
rm_meanRT = AnovaRM(mean_RT, 'mainEstimateRT', 'ID', within=['RunNumber'])
res_meanRT = rm_meanRT.fit()
# print(res_meanRT.summary())

In [ ]:
#oncet of RTs

# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import statsmodels.api as sm  # For statistical models
from statsmodels.formula.api import ols, mixedlm  # For Ordinary Least Squares and Mixed Linear Models
from statsmodels.stats.anova import AnovaRM  # For Repeated Measures ANOVA
from statsmodels.stats.multicomp import pairwise_tukeyhsd  # For Tukey's HSD test
from scipy.stats import ttest_rel, ttest_ind  # For paired and independent t-tests
from statsmodels.stats.multitest import multipletests  # For multiple testing correction
import pingouin as pg  # For statistical tests and data analysis

# Group the data by participant (ID) and condition (Run number), and calculate the mean of the response (RT) for each group
mean_RT = df_concat.groupby(['ID', 'RunNumber'])['reactionOnset'].mean().reset_index()

# Pivot the data to create a matrix of means, with participants as rows and conditions as columns
# mean_RT_matrix = mean_RT.pivot(index='ID', columns='RunNumber', values='mainEstimateRT')

# Set the color palette for the plots to "rocket"
sns.set_palette("rocket")

# Create a violin plot to visualize the distribution of means of RTs by condition
ax = sns.violinplot(x='RunNumber', y='reactionOnset', data=mean_RT)

# Add point plot to the violin plot to show mean RT with error bars representing standard deviation
sns.pointplot(x='RunNumber', y='reactionOnset', hue='ID', errorbar='sd', data=mean_RT,
              dodge=False, err_kws={'linewidth': 1}, linewidth=1, palette="rocket", ax=ax)

# Perform pairwise t-tests between conditions to assess differences in RTs
pairwise_res_mean_RT = pg.pairwise_tests(data=mean_RT, dv='reactionOnset', within='RunNumber', subject='ID')

# # Loop through the results of pairwise tests to add significance indicators to the plot
for i in range(len(pairwise_res_mean_RT)):
    if pairwise_res_mean_RT['p-unc'][i] < 0.05:  # Check if the p-value is less than the significance level (0.05)
        # Calculate the x-coordinates for the lines indicating significant differences
        x_min = pairwise_res_mean_RT['A'][i]
        x_min = (x_min - 1) / 1  # Normalize x_min for plotting
        x_max = pairwise_res_mean_RT['B'][i]
        x_max = (x_max - 1) / 1  # Normalize x_max for plotting
        
        # Set the y-coordinate for the significance line
        y_line =  0.25*i + 2.5
        p_val = round(pairwise_res_mean_RT['p-unc'][i], 2)  # Round p-value for display
        
        # Draw a horizontal line to indicate significant differences
        ax.hlines(y=y_line, xmin=x_min, xmax=x_max, color='k', linewidth=1)
        
        # Annotate the plot with the p-value
        ax.annotate("p = {}".format(p_val), xy=((x_min + x_max) * .5, y_line), 
                    ha='center', va='bottom', color='k', fontsize=10)

# Remove top and right spines from the plot for better aesthetics
sns.despine()

# Set the limits and labels for the plot
plt.title('onset of RT across runs\n', fontsize=18)
plt.xlabel('run number', fontsize=16)
plt.ylabel('mean RT', fontsize=16)

# Remove legend labels for clarity
ax.legend(labels=[])

# Save the figure to a file with high resolution
plt.savefig('./Figures/' + 'onset_RT_across_runs.png', bbox_inches="tight", dpi=300)

# Display the plot
plt.show()

# perform a repeated measures ANOVA on the mean RT of the response for each condition
rm_meanRT = AnovaRM(mean_RT, 'reactionOnset', 'ID', within=['RunNumber'])
res_meanRT = rm_meanRT.fit()
print(res_meanRT.summary())

In [ ]:
for df_temp in df_no_missed_no_catch_list:
    mean_value = np.mean(df_temp.iloc[0:23,11].values.reshape(-1, 1)).item()
    
    rvalues=(df_temp.iloc[0:23,11].values.reshape(-1, 1))
    # rvalues.reshape(-1)
    # print(*rvalues)
    print(mean_value)

In [ ]:
# for df_temp in df_no_missed_no_catch_list:
mean_value = np.mean(df_no_missed_no_catch_list[1].iloc[:,20].values.reshape(-1, 1)).item()

rvalues=(df_no_missed_no_catch_list[1].iloc[:,20].values.reshape(-1, 1))
est_values=df_no_missed_no_catch_list[1].iloc[:,9].values.reshape(-1, 1)
# rvalues.reshape(-1)
print(*rvalues)
print(*est_values)
print(mean_value)

In [ ]:
from statsmodels.formula.api import mixedlm
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests 

means_data = df_concat.groupby(['ID', 'RunNumber'])['est_error'].mean().reset_index()

# set the color palette to "rocket"
sns.set_palette("rocket")

ax=sns.violinplot(x='RunNumber', y='est_error', data=means_data, inner=None)

# # add swarmplot points to the violin plot
# sns.swarmplot(x='RunNumber', y='center_dist', data=means_data, hue='ID', palette='dark:white', edgecolor='auto', dodge=False, ax=ax)
sns.pointplot(x='RunNumber', y='est_error', hue='ID', errorbar='sd', data=means_data, \
                   dodge=False, err_kws={'linewidth': 1}, linewidth=1, palette="mako", ax=ax)

plt.title('')
plt.xlabel('run')
plt.ylabel('estimation error')
# ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
# deactivate the legend
ax.legend(labels=[])
plt.show()



# perform a repeated measures ANOVA on the standard deviation of the response for each condition
rm = AnovaRM(means_data, 'est_error', 'ID', within=['RunNumber'])
res = rm.fit()
print(res.summary())

# add significance markers to the plot
tukey_center_mean_results = pairwise_tukeyhsd(means_data['est_error'], means_data['RunNumber'], alpha=0.05)
tukey_center_mean_results = pd.DataFrame(data=tukey_center_mean_results._results_table.data[1:], columns=tukey_center_mean_results._results_table.data[0])
tukey_center_mean_results = tukey_center_mean_results[tukey_center_mean_results['reject'] == True]
# y_values = [std_dev['CenterDist'].max()*1.1, std_dev['CenterDist'].max()*1.2, std_dev['CenterDist'].max()*1.3]


print(tukey_center_mean_results)
# for i, row in tukey_center_mean_results.iterrows():
#     x1, x2 = std_dev['RunNumber'].unique().tolist().index(row['group1']), std_dev['RunNumber'].unique().tolist().index(row['group2'])
#     y = y_values[i%3]
#     pval = round(row['p-adj'], 3)
#     ax.hlines(y=y, xmin=x1, xmax=x2, color='k', linewidth=1)
#     ax.annotate("p = {}".format(pval), xy=((x1+x2)*.5, y), ha='center', va='bottom', color='k', fontsize=10)


# Perform pairwise t-tests between conditions
pairwise_res = pg.pairwise_tests(data=means_data, dv='est_error', within='RunNumber', subject='ID')

# Add significance indicators to the plot
for i in range(len(pairwise_res)):
    if pairwise_res['p-unc'][i] < 0.05:  # Adjust significance level as needed
        x_min=pairwise_res['A'][i]
        x_min=(x_min-120)/120
        x_max=pairwise_res['B'][i]
        x_max=(x_max-120)/120
        y_line=10*i+140
        p_val=round(pairwise_res['p-unc'][i], 2)
        print(p_val)
        # zero_line=np.zeros(len(xmax-xmin))
        # sns.lineplot(x=[xmin, xmax], y=[4*i+120, 4*i+120], color='m', label=round(pairwise_res['p-unc'][i],2), linestyle='--', ax=ax)

        # ax.hlines(y=y_line, xmin=x_min, xmax=x_max, color='k', linewidth=1)
        # ax.annotate("p = {}".format(p_val), xy=((x_min+x_max)*.5, y_line), ha='center', va='bottom', color='k', fontsize=10)



In [ ]:

std_center_data = df_concat.groupby(['ID', 'RunNumber'])['CenterDist'].std().reset_index()

# set the color palette to "rocket"
sns.set_palette("rocket")

ax=sns.violinplot(x='RunNumber', y='CenterDist', data=std_center_data)

# add swarmplot points to the violin plot
sns.swarmplot(x='RunNumber', y='CenterDist', data=std_center_data, hue='ID', color='white', edgecolor='gray', ax=ax)


plt.xlabel('Run')
plt.ylabel('SD response distance from the center')
# plt.title('Participant Response Across Runs')
# plt.legend(title='Participant')
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
# deactivate the legend
ax.legend(labels=[])
plt.show()

# perform a repeated measures ANOVA on the standard deviation of the response for each condition
rm_sd = AnovaRM(std_center_data, 'CenterDist', 'ID', within=['RunNumber'])
res_sd = rm_sd.fit()
print(res_sd.summary())

# add significance markers to the plot
tukey_center_sd_results = pairwise_tukeyhsd(std_center_data ['CenterDist'], std_center_data ['RunNumber'], alpha=0.05)
tukey_center_sd_results = pd.DataFrame(data=tukey_center_sd_results._results_table.data[1:], columns=tukey_center_sd_results._results_table.data[0])
tukey_center_sd_results = tukey_center_sd_results[tukey_center_sd_results['reject'] == True]
# y_values = [std_dev['CenterDist'].max()*1.1, std_dev['CenterDist'].max()*1.2, std_dev['CenterDist'].max()*1.3]


print(tukey_center_sd_results)


In [ ]:
from scipy.stats import linregress, t, ttest_1samp

import pandas as pd
import numpy as np

df_slope = df_concat.copy()

# Calculate the slopes of the fitted lines for each participant
slopes = []
for participant in df_slope['ID'].unique():
    estimation = df_slope.loc[df_slope['ID'] == participant, 'mainEstimate']
    actual = df_slope.loc[df_slope['ID'] == participant, 'mainMeters']
    slope, _, _, _, _ = linregress(estimation, actual)
    slopes.append(slope)

print (slopes)
# Calculate the standard deviation of the slopes
std_dev_slopes = np.std(slopes)

# Calculate the confidence interval for the standard deviation
n = len(slopes)
t_value = t.ppf(0.975, n - 1)
std_error = std_dev_slopes / np.sqrt(n)
ci_low = std_dev_slopes - t_value * std_error
ci_high = std_dev_slopes + t_value * std_error

print("Standard deviation:", std_dev_slopes)
print("95% Confidence interval:", (ci_low, ci_high))
print('\n')

# Perform a one-sample t-test to determine if the standard deviation is significant
t_stat, p_val = ttest_1samp(slopes, 0)

print(p_val)
print('\n')

if p_val < 0.05:
    print("The variability in responses across subjects is statistically significant.")
else:
    print("The variability in responses across subjects is not statistically significant.")
print('\n')